# notebook for determining FE55 gains and camera charge diffusion

Eric Charles and Bo Xin

References:

    o. https://github.com/lsst/bootcamp-work/blob/master/examples/welcome_to_FE55.ipynb

    o. https://github.com/lsst-camera-dh/eotest/blob/master/python/lsst/eotest/sensor/fe55_psf.py

### Setup

Before running this notebook, you'll need to set up the `obs_lsst` package. Before doing the setup procedure below, you might want to run the notebook until it crashes so you see what the error message looks like.

Step-by-step instructions:

1. Start a terminal in JupyterLab. In the terminal, setup the Stack with the command `source /opt/lsst/software/stack/loadLSST.bash` and then issue the command `setup lsst_distrib` to allow you to run scons in a subsequent step.

2. Create and/or switch into a folder where you want to put your local versions of the LSST Stack (e.g., `~/repos`)

Run the following commands

```
git clone https://github.com/lsst/obs_lsstCam.git
cd obs_lsstCam
setup -j -r .
scons
```

3. Add `setup -k -r path_to_repos/obs_lsstCam` to `$HOME/notebooks/.user_setups`.

4. Restart your kernel.

Just for fun, check what version of the Stack you are using. This notebook has been tested on `w_2018_45`.

### Imports

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

# system imports
from matplotlib import pylab as plt
from scipy.optimize import leastsq
import numpy
import os
import yaml


# Firefly client imports
from firefly_client import FireflyClient
from IPython.display import IFrame

import stats_utils
import table_utils
from scipy.special import erf, gammaincc
from astropy.table import Table

In [ ]:
import FE55_psf_func
import stats_utils
import table_utils
import plot_utils

In [ ]:
infile_base = "FE55_clusters"
outfilehist_base = "FE55_summary"

bin_dict = dict(sigmax=(0.25, 0.75, 50),
                sigmay=(0.25, 0.75, 50),
                dn=(2000, 3000., 100),
                x0=(None, None, 100),
                y0=(None, None, 100),
                dn_fp=(2000, 3500, 100),
                chiprob=(0., 1., 100),
                chi2=(None, None, 100),
                dof=(0, 50, 50))
                                
gauss_fits = ['sigmax', 'sigmay']
double_gauss_fits = ['dn', 'dn_fp']
ccd_hists_list = []
for iamp in range(16):
    infile = "%s_%02i.fits" % (infile_base, iamp)
    intable = Table.read(infile)
    
    ccd_hists = stats_utils.bin_lists(intable, bin_dict)
    stats_utils.fit_hists_gaussian(ccd_hists, gauss_fits)
    stats_utils.fit_hists_double_gaussian(ccd_hists, double_gauss_fits)

    ccd_hists_list.append(ccd_hists)
    
    if outfilehist_base is None:
        outstream = sys.stdout
    else:
        outstream = open("%s_%02i.yaml" %(outfilehist_base, iamp), 'w')
    yaml.dump(ccd_hists, outstream)
    if outfilehist_base is not None:
        outstream.close()
    

### Set up the necessary classes for using firefly to look at the data.

### plot distributions for all 16 amps.

In [ ]:
my_channel = '{}_test_channel'.format(os.environ['USER'])
server = 'https://lsst-lspdev.ncsa.illinois.edu'


ff='{}/firefly/slate.html?__wsch={}'.format(server, my_channel)
IFrame(ff,800,600)

In [ ]:
fig, axs = plot_utils.plot_all_amps(ccd_hists_list, 'sigmax',
                                    xlabel='sigma x', ylabel='N clusters', savepng='sigmax.png')

In [ ]:
fig, axs = plot_utils.plot_all_amps(ccd_hists_list, 'sigmay',
                                    xlabel='sigma y', ylabel='N clusters', savepng='sigmay.png')

In [ ]:
fig, axs = plot_utils.plot_all_amps(ccd_hists_list, 'dn',
                                    xlabel='n[e]', ylabel='N clusters', savepng='dn.png')

In [ ]:
fig, axs = plot_utils.plot_all_amps(ccd_hists_list, 'dn_fp',
                                    xlabel=r'$n_{foot}$[e]', ylabel='N clusters', savepng='dn.png')

In [ ]:
fig, axs = plot_utils.plot_all_amps(ccd_hists_list, 'chiprob',
                                    xlabel='P($\chi^2)', ylabel='N clusters', savepng='chiprob.png')

In [ ]:
FE55_electrons = 1594
gain_clusterFit = numpy.zeros((16,1))
for i in range(16):
    gain_clusterFit[i] = FE55_electrons/ccd_hists_list[i]['dn']['fit'][0][1]
    print("Gain for amp %02i -- %.5f"%(i, gain_clusterFit[i]))

In [ ]:
# gain measurements from Simon's demo notebook
gain_demo = [
    0.68561, 0.68284, 0.68053, 0.68318, 
    0.68491, 0.67897, 0.68294, 0.68767, 
    0.68107, 0.67561, 0.67900, 0.67693, 
    0.67826, 0.67734, 0.67541, 0.67207]

In [ ]:
xp=numpy.arange(16)
plt.plot(xp, gain_clusterFit, 'ro', label = 'Gains from fitting to FE55 clusters')
plt.plot(xp, gain_demo, 'b*', label = 'Gains from Simon demo notebook')
plt.legend()